# Global stress objective example

## Commented Program

What follows is a program spliced with comments.

In [1]:
using TopOpt, LinearAlgebra

### Define the problem

In [2]:
E = 1.0 # Young’s modulus
v = 0.3 # Poisson’s ratio
f = 1.0 # downward force
rmin = 3.0 # filter radius

problems = Any[
    PointLoadCantilever(Val{:Linear}, (60, 20), (1.0, 1.0), E, v, f),
    HalfMBB(Val{:Linear}, (60, 20), (1.0, 1.0), E, v, f),
]
problem_names = ["Cantilever beam", "Half MBB beam", "L-beam", "Tie-beam"]

i = 1
println(problem_names[i])
problem = problems[i]

Cantilever beam


### Parameter settings

In [3]:
V = 0.5 # volume fraction
xmin = 0.001 # minimum density
steps = 40 # maximum number of penalty steps, delta_p0 = 0.1
convcriteria = Nonconvex.KKTCriteria()
penalty = TopOpt.PowerPenalty(1.0)

PowerPenalty{Float64}(1.0)

### Define a finite element solver

In [4]:
solver = FEASolver(Direct, problem; xmin=xmin, penalty=penalty)

### Define **stress** objective
Notice that gradient is derived automatically by automatic differentiation (Zygote.jl)!

In [5]:
stress = TopOpt.von_mises_stress_function(solver)
filter = if problem isa TopOptProblems.TieBeam
    identity
else
    DensityFilter(solver; rmin=rmin)
end
volfrac = TopOpt.Volume(solver)

x0 = ones(length(solver.vars))
threshold = 3 * maximum(stress(filter(PseudoDensities(x0))))

obj = x -> volfrac(filter(PseudoDensities(x)))
constr = x -> norm(stress(filter(PseudoDensities(x))), 5) - threshold

#3 (generic function with 1 method)

### Define subproblem optimizer

In [6]:
N = length(solver.vars)
x0 = fill(0.5, N)

options = MMAOptions(; maxiter=2000, tol=Nonconvex.Tolerance(; kkt=1e-4))
model = Model(obj)
addvar!(model, zeros(N), ones(N))
add_ineq_constraint!(model, constr)
alg = MMA87()
r = optimize(model, alg, x0; options, convcriteria)

@show obj(r.minimizer)
@show constr(r.minimizer)

Internal error: encountered unexpected error in runtime:
TypeError(func=:<:, context="", expected=Type, got=TApprox<:NonconvexMMA.AbstractMMAApprox)
jl_type_error_rt at /buildworker/worker/package_linux64/build/src/rtutils.c:119
jl_type_error at /buildworker/worker/package_linux64/build/src/rtutils.c:127
jl_f_issubtype at /buildworker/worker/package_linux64/build/src/builtins.c:491
apply_type_nothrow at ./compiler/tfuncs.jl:1245
_builtin_nothrow at ./compiler/tfuncs.jl:1566
stmt_effect_free at ./compiler/ssair/queries.jl:36
check_effect_free! at ./compiler/ssair/inlining.jl:1128 [inlined]
process_simple! at ./compiler/ssair/inlining.jl:1185
assemble_inline_todo! at ./compiler/ssair/inlining.jl:1358
ssa_inlining_pass! at ./compiler/ssair/inlining.jl:80
jfptr_ssa_inlining_passNOT._15146.clone_1 at /opt/hostedtoolcache/julia/1.7.3/x64/lib/julia/sys.so (unknown line)
_jl_invoke at /buildworker/worker/package_linux64/build/src/gf.c:2247 [inlined]
jl_apply_generic at /buildworker/worker/pack

-0.00025090938076743896

### (Optional) Visualize the result using Makie.jl
Need to run `using Pkg; Pkg.add(Makie)` first
```julia
using TopOpt.TopOptProblems.Visualization: visualize
fig = visualize(problem; topology = r.minimizer)
Makie.display(fig)
```

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*